In Class 6 - Dario Placencio

In [3]:
# Import the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import nltk
# normalization
import unicodedata
# stopwords
from nltk.corpus import stopwords
# punctuation
import string
# clean using regex expression
import re
#stemming
from nltk.stem.snowball import SnowballStemmer
# lemmatize
from nltk.stem import WordNetLemmatizer

In [4]:
# nltk.download('stopwords')
sw = set(stopwords.words("english"))

In [5]:
# define stemmer
stemmer = SnowballStemmer("english")

In [2]:
# Read the workers compensation data set

dat = pd.read_csv("WorkersComp.csv") 
dat.head()

,ClaimNumber,DateTimeOfAccident,DateReported,Age,Gender,MaritalStatus,DependentChildren,DependentsOther,WeeklyWages,PartTimeFullTime,HoursWorkedPerWeek,DaysWorkedPerWeek,ClaimDescription,InitialIncurredCalimsCost,UltimateIncurredClaimCost
0,WC8285054,2002-04-09T07:00:00Z,2002-07-05T00:00:00Z,48,M,M,0,0,500.00,F,38.0,5,LIFTING TYRE INJURY TO RIGHT ARM AND WRIST INJURY,1500,4748.203388
1,WC6982224,1999-01-07T11:00:00Z,1999-01-20T00:00:00Z,43,F,M,0,0,509.34,F,37.5,5,STEPPED AROUND CRATES AND TRUCK TRAY FRACTURE ...,5500,6326.285819
2,WC5481426,1996-03-25T00:00:00Z,1996-04-14T00:00:00Z,30,M,U,0,0,709.10,F,38.0,5,CUT ON SHARP EDGE CUT LEFT THUMB,1700,2293.949087
3,WC9775968,2005-06-22T13:00:00Z,2005-07-22T00:00:00Z,41,M,S,0,0,555.46,F,38.0,5,DIGGING LOWER BACK LOWER BACK STRAIN,15000,17786.487170
4,WC2634037,1990-08-29T08:00:00Z,1990-09-27T00:00:00Z,36,M,M,0,0,377.10,F,38.0,5,REACHING ABOVE SHOULDER LEVEL ACUTE MUSCLE STR...,2800,4014.002925


In [6]:
# Remove stop words from Claim Description  
dat['ClaimDescription'] = dat['ClaimDescription'].apply(lambda x: ' '.join([word for word in x.split() if word not in (sw)]))

In [7]:
Stem = []
for Claim in dat.ClaimDescription:
    text = Claim.lower() # lowerlize
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = text.split()  # plit
    tokens=[stemmer.stem(i) for i in text if (i not in string.punctuation)&(i not in sw)]
    Stem.append(tokens)

In [8]:
dat['Stem'] = Stem

In [9]:
Stem[0:5]

[['lift', 'tyre', 'injuri', 'right', 'arm', 'wrist', 'injuri'],
 ['step', 'around', 'crate', 'truck', 'tray', 'fractur', 'left', 'forearm'],
 ['cut', 'sharp', 'edg', 'cut', 'left', 'thumb'],
 ['dig', 'lower', 'back', 'lower', 'back', 'strain'],
 ['reach',
  'shoulder',
  'level',
  'acut',
  'muscl',
  'strain',
  'left',
  'side',
  'stomach']]

In [10]:
#nltk.download('wordnet')
#nltk.download('omw-1.4')
# define lemmatizer
lemmatizer = WordNetLemmatizer()

In [11]:
Lemm = []
lemmatizer = WordNetLemmatizer()
for Claim in dat.Stem:
    #text = Claim.lower()
    #text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    #text = re.sub(r'https?://\S+|www\.\S+', '', text)
    #text = text.split()
    tokens=[lemmatizer.lemmatize(i) for i in Claim] #if (i not in string.punctuation)&(i not in sw)]
    Lemm.append(tokens)

In [12]:
Lemm[0:5]

[['lift', 'tyre', 'injuri', 'right', 'arm', 'wrist', 'injuri'],
 ['step', 'around', 'crate', 'truck', 'tray', 'fractur', 'left', 'forearm'],
 ['cut', 'sharp', 'edg', 'cut', 'left', 'thumb'],
 ['dig', 'lower', 'back', 'lower', 'back', 'strain'],
 ['reach',
  'shoulder',
  'level',
  'acut',
  'muscl',
  'strain',
  'left',
  'side',
  'stomach']]

### Unigram Model

In [13]:
### N-grams

from nltk import ngrams

In [14]:
# define function for uni-gram
def uni(tokens):
    return tokens

dat['Stem_uni'] = dat['Stem'].apply(lambda x: uni(x)) # uni-gram

In [15]:
# Preview the first 5 rows of the Stem_uni column
dat['Stem_uni'].head()

0      [lift, tyre, injuri, right, arm, wrist, injuri]
1    [step, around, crate, truck, tray, fractur, le...
2                  [cut, sharp, edg, cut, left, thumb]
3              [dig, lower, back, lower, back, strain]
4    [reach, shoulder, level, acut, muscl, strain, ...
Name: Stem_uni, dtype: object

1. This is a continuation of Rlab #5. Consider data set "WorkersComp.csv". Use the unigram from Rlab #5 to build a topic model with 3 topics.

a) Report the top 10 most probable words for each topic.

b) Report the topic distribution for 3rd, 5th, and 9th claims

In [16]:
# Use the uni-gram to build a topic model with 3 topics
from gensim import corpora, models

# Create Dictionary
id2word = corpora.Dictionary(dat['Stem_uni'])

# Create Corpus
texts = dat['Stem_uni']

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# Build LDA model
lda_model = models.LdaModel(corpus=corpus,  # corpus
                            id2word=id2word, # id to word
                            num_topics=3,   # number of topics
                            random_state=100, # random state
                            chunksize=100, # chunk size
                            passes=10, # number of passes
                            alpha='auto', # auto alpha
                            per_word_topics=True) # per word topic

# Print the Keyword in the 3 topics
print(lda_model.print_topics())

[(0, '0.175*"strain" + 0.144*"back" + 0.088*"lower" + 0.077*"lift" + 0.051*"shoulder" + 0.032*"neck" + 0.025*"injuri" + 0.017*"pain" + 0.014*"floor" + 0.014*"fell"'), (1, '0.135*"right" + 0.064*"eye" + 0.058*"left" + 0.058*"knee" + 0.047*"foreign" + 0.046*"bodi" + 0.027*"foot" + 0.021*"burn" + 0.018*"tendon" + 0.016*"wrist"'), (2, '0.107*"left" + 0.078*"right" + 0.069*"lacer" + 0.058*"finger" + 0.055*"hand" + 0.049*"struck" + 0.026*"cut" + 0.026*"fell" + 0.024*"thumb" + 0.023*"bruis"')]


In [17]:
# Print the top 10 most probable words for each of the 3 topics
lda_model.show_topics(num_topics=3, num_words=10)

[(0,
  '0.175*"strain" + 0.144*"back" + 0.088*"lower" + 0.077*"lift" + 0.051*"shoulder" + 0.032*"neck" + 0.025*"injuri" + 0.017*"pain" + 0.014*"floor" + 0.014*"fell"'),
 (1,
  '0.135*"right" + 0.064*"eye" + 0.058*"left" + 0.058*"knee" + 0.047*"foreign" + 0.046*"bodi" + 0.027*"foot" + 0.021*"burn" + 0.018*"tendon" + 0.016*"wrist"'),
 (2,
  '0.107*"left" + 0.078*"right" + 0.069*"lacer" + 0.058*"finger" + 0.055*"hand" + 0.049*"struck" + 0.026*"cut" + 0.026*"fell" + 0.024*"thumb" + 0.023*"bruis"')]

In [18]:
# Report the topic distribution for the 3rd, the 5th, and the 9th claim
for i in [3,5,9]:
    print(lda_model.get_document_topics(corpus[i]))

[(0, 0.92666584), (1, 0.026386414), (2, 0.046947755)]
[(0, 0.041167222), (1, 0.030991953), (2, 0.9278408)]
[(0, 0.03149894), (1, 0.73781335), (2, 0.23068772)]


2. Consider word embeedings from 'glove-twitter-25'.

a) Report the embedding vectors for words "tea", "phone", "coffee", and "moon".

b) Calculate the similarity matrix, and identify the pair with highest similarity.


In [19]:
import gensim.downloader as api
# information about the names of pre-trained models
api.info(name_only=True)

{'corpora': ['semeval-2016-2017-task3-subtaskBC',
  'semeval-2016-2017-task3-subtaskA-unannotated',
  'patent-2017',
  'quora-duplicate-questions',
  'wiki-english-20171001',
  'text8',
  'fake-news',
  '20-newsgroups',
  '__testing_matrix-synopsis',
  '__testing_multipart-matrix-synopsis'],
 'models': ['fasttext-wiki-news-subwords-300',
  'conceptnet-numberbatch-17-06-300',
  'word2vec-ruscorpora-300',
  'word2vec-google-news-300',
  'glove-wiki-gigaword-50',
  'glove-wiki-gigaword-100',
  'glove-wiki-gigaword-200',
  'glove-wiki-gigaword-300',
  'glove-twitter-25',
  'glove-twitter-50',
  'glove-twitter-100',
  'glove-twitter-200',
  '__testing_word2vec-matrix-synopsis']}

In [20]:
# Get pretrained 25 dimension embeddings from twitter data using glove
model3 = api.load("glove-twitter-25")

[==================================================] 100.0% 104.8/104.8MB downloaded


In [21]:
model3.most_similar("tea")  # show words that similar to word 'tea'

[('coffee', 0.917603075504303),
 ('milk', 0.905985414981842),
 ('cream', 0.902299702167511),
 ('bread', 0.9010369777679443),
 ('ice', 0.8975890874862671),
 ('wine', 0.8962970972061157),
 ('food', 0.8928905725479126),
 ('soup', 0.8913701176643372),
 ('cake', 0.8854183554649353),
 ('cheese', 0.8838366270065308)]

In [22]:
model3.most_similar("phone")  # show words that similar to word 'phone'

[('text', 0.8994132876396179),
 ('id', 0.8980249762535095),
 ('screen', 0.8937858939170837),
 ('line', 0.8775232434272766),
 ('bc', 0.8747655153274536),
 ('call', 0.8741815090179443),
 ('phones', 0.8709824681282043),
 ('number', 0.8700169920921326),
 ('send', 0.8674761056900024),
 ('contacts', 0.8586106896400452)]

In [23]:
model3.most_similar("coffee")  # show words that similar to word 'coffee'

[('wine', 0.9737642407417297),
 ('food', 0.9210957884788513),
 ('bath', 0.9208705425262451),
 ('tea', 0.9176030158996582),
 ('cream', 0.9174261689186096),
 ('ice', 0.9166433215141296),
 ('beer', 0.9165878891944885),
 ('fresh', 0.9100037217140198),
 ('breakfast', 0.9091862440109253),
 ('drink', 0.9059524536132812)]

In [24]:
model3.most_similar("moon")  # show words that similar to word 'moon'

[('sun', 0.8894006609916687),
 ('ocean', 0.8843007683753967),
 ('soul', 0.8822776675224304),
 ('earth', 0.8816424012184143),
 ('shadow', 0.8800047636032104),
 ('shining', 0.8745366334915161),
 ('stone', 0.8728160262107849),
 ('light', 0.8710013628005981),
 ('heaven', 0.8636651039123535),
 ('joy', 0.8595615029335022)]

In [25]:
# Calculate the similarity between the four words, and identify the most similar pair
print(model3.similarity("tea", "coffee"))
print(model3.similarity("tea", "phone"))
print(model3.similarity("tea", "moon"))
print(model3.similarity("coffee", "phone"))
print(model3.similarity("coffee", "moon"))
print(model3.similarity("phone", "moon"))

0.91760314
0.506179
0.56842726
0.5559021
0.65578926
0.50199217


### The most similar pair is 'tea' and 'coffee'